In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine
import yaml

mysql_info = yaml.safe_load(open('mysql.yml'))

# 요청에 필요한 데이터 설정
url = 'https://statiz.sporki.com/stats/?m=team&m2=batting&m3=default&so=&ob=&year=2024&sy=2019&ey=2024&te=&po=&lt=10100&reg=&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=1000&ph=&hs=&us=&na=&ls=1&sf1=WAROff&sk1=&sv1=&sf2=WAROff&sk2=&sv2='
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

res = requests.get(url, headers=headers)
res.raise_for_status()

html = response.text

# 응답에서 데이터 추출
soup = BeautifulSoup(html, "html.parser")

rows = []
data = []

trs = soup.find_all("tr")

for tr in trs[10:]:
    tds = tr.find_all("td")
    team_id = tds[1].find('a').attrs['href'].split('/')[2].split('&')[1].split('=')[1] 
    tds = [td.text for td in tds]
    tds.append(team_id)
    rows.append(tds)

for row in rows:
    temp = []
    for index in range(1, len(row)):
        temp.append(row[index])
    data.append(temp)

In [3]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

team_hitter_df = pd.DataFrame(data, columns=['팀', '시즌', 'WAR', 'oWAR', 'dWAR', 'G', 'PA', 'ePA', 'AB', 'R', 'H', '2B', '3B', 'HR', 'TB', 'RBI', 'SB', 'CS', 'BB', 'HP', 'IB', 'SO', 'GDP', 'SH', 'SF', 'AVG', 'OBP', 'SLG', 'OPS', 'R/ePA', 'WRC+', 'DUMMY', 'T_CODE'])

team_hitter_df = team_hitter_df.dropna(axis=0)

team_hitter_df.head()

,팀,시즌,WAR,oWAR,dWAR,G,PA,ePA,AB,R,H,2B,3B,HR,TB,RBI,SB,CS,BB,HP,IB,SO,GDP,SH,SF,AVG,OBP,SLG,OPS,R/ePA,WRC+,DUMMY,T_CODE
0,두산,19,40.29,35.15,5.14,1738,5670,5622,4913,736,1364,235,31,84,1913,691,102,38,563,72,16,799,117,39,83,0.278,0.355,0.389,0.744,0.131,114.5,40.29,6002
1,NC,20,38.36,35.30,3.06,1978,5833,5772,5102,888,1483,258,28,187,2358,845,101,38,529,104,21,997,106,46,51,0.291,0.366,0.462,0.828,0.154,113.7,38.36,11001
2,LG,22,37.88,34.17,3.70,1735,5627,5566,4927,715,1325,208,33,118,1953,674,102,46,501,103,18,924,109,49,46,0.269,0.346,0.396,0.742,0.129,113.8,37.88,5002
3,LG,23,37.34,36.56,0.78,1735,5687,5587,4881,767,1364,227,27,93,1924,714,166,101,583,73,16,804,89,94,55,0.279,0.361,0.394,0.755,0.137,119.6,37.34,5002
4,KT,20,35.33,30.83,4.50,2002,5762,5687,5047,813,1432,238,21,163,2201,767,106,50,554,52,20,1097,104,64,45,0.284,0.358,0.436,0.794,0.143,108.6,35.33,12001


In [6]:
if_exists = 'replace'
# 데이터베이스 계정과 사용자를 사용하여 Snowflake에 연결 생성

account_identifier = mysql_info['account_identifier']
user = mysql_info['user']
password = mysql_info['password']
conn_string = f"mysql://{user}:{password}@{account_identifier}:3306/baseball"
engine = create_engine(conn_string)

table_name = 'df_team_hitter'

with engine.connect() as con:
    team_hitter_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )

In [14]:
# 요청에 필요한 데이터 설정
url = 'https://statiz.sporki.com/stats/?m=team&m2=pitching&m3=default&so=&ob=&year=2024&sy=2019&ey=2024&te=&po=&lt=10100&reg=&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=1000&ph=&hs=&us=&na=&ls=1&sf1=G&sk1=&sv1=&sf2=G&sk2=&sv2='

response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

res = requests.get(url, headers=headers)
res.raise_for_status()

html = response.text

# 응답에서 데이터 추출
soup = BeautifulSoup(html, "html.parser")

rows = []
data = []

trs = soup.find_all("tr")

for tr in trs[10:]:
    tds = tr.find_all("td")
    team_id = tds[1].find('a').attrs['href'].split('&')[1].split('=')[1]
    tds = [td.text for td in tds]
    tds.append(team_id)
    rows.append(tds)

for row in rows:
    temp = []
    for index in range(1, len(row)):
        temp.append(row[index])
    data.append(temp)

In [17]:
team_pitcher_df = pd.DataFrame(data, columns=['팀', '시즌', 'WAR', 'G', 'GS', 'GR', 'GF', 'CG', 'SHO', 'W', 'L', 'S', 'HD', 'IP', 'ER', 'R', 'rRA', 'TBF', 'H', '2B', '3B', 'HR', 'BB', 'HP', 'IB', 'SO', 'ROE', 'BK', 'WP', 'ERA', 'RA9', 'rRA9', 'FIP', 'WHIP', 'DUMMY', 'T_CODE'])

team_pitcher_df = team_pitcher_df.dropna(axis=0)

team_pitcher_df.head()

,팀,시즌,WAR,G,GS,GR,GF,CG,SHO,W,L,S,HD,IP,ER,R,rRA,TBF,H,2B,3B,HR,BB,HP,IB,SO,ROE,BK,WP,ERA,RA9,rRA9,FIP,WHIP,DUMMY,T_CODE
0,SK,19,36.41,628,144,484,144,0,0,88,55,51,92,1292.1,500,546,546.00,5463,1240,176,14,99,436,50,12,1092,46,4,50,3.48,3.80,3.80,3.74,1.30,36.41,9001
1,KT,21,29.04,614,144,470,144,2,2,76,59,33,74,1264.0,515,584,584.00,5397,1176,184,13,86,489,42,18,1062,79,1,56,3.67,4.16,4.16,3.66,1.32,29.04,12001
2,KT,22,28.19,588,144,444,144,1,1,80,62,37,65,1287.1,502,562,562.00,5405,1243,188,19,92,364,63,22,1074,56,3,43,3.51,3.93,3.93,3.52,1.25,28.19,12001
3,키움,19,27.80,611,144,467,144,2,2,86,57,39,91,1279.0,513,572,572.00,5427,1293,244,24,61,371,58,3,984,59,6,45,3.61,4.03,4.03,3.44,1.30,27.80,10001
4,LG,21,27.40,697,144,553,144,0,0,72,58,32,81,1272.1,505,561,561.00,5453,1126,197,18,80,545,98,18,1068,59,5,43,3.57,3.97,3.97,3.88,1.31,27.40,5002
5,삼성,21,26.96,675,143,532,144,2,1,76,59,46,80,1259.0,601,658,658.00,5536,1297,230,24,131,530,51,13,1038,49,3,58,4.30,4.70,4.70,4.22,1.45,26.96,1001
6,KIA,20,26.93,639,144,495,144,1,1,73,71,33,79,1270.0,724,795,795.00,5681,1399,265,34,120,559,70,8,1038,66,4,79,5.13,5.63,5.63,4.50,1.54,26.93,2002
7,롯데,23,25.43,691,144,547,144,1,0,68,76,35,82,1272.1,586,660,660.00,5653,1328,223,24,80,532,82,20,1070,65,2,71,4.15,4.67,4.67,3.90,1.46,25.43,3001
8,LG,22,25.11,748,144,604,144,0,0,87,55,43,107,1288.0,476,521,521.00,5432,1179,166,34,94,451,59,27,1031,60,0,49,3.33,3.64,3.64,3.82,1.27,25.11,5002
9,키움,22,24.81,649,144,505,144,2,1,80,62,48,84,1296.1,546,610,610.00,5579,1247,259,15,96,490,70,18,1010,77,1,51,3.79,4.24,4.24,4.03,1.34,24.81,10001


In [18]:
table_name = 'df_team_pitcher'

with engine.connect() as con:
    team_pitcher_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )

In [20]:
url = 'https://statiz.sporki.com/stats/?m=team&m2=fielding&m3=default&so=&ob=&year=2024&sy=2019&ey=2024&te=&po=&lt=&reg=&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=1000&ph=&hs=&us=&na=&ls=1&sf1=G&sk1=&sv1=&sf2=G&sk2=&sv2='

response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

res = requests.get(url, headers=headers)
res.raise_for_status()

html = response.text

# 응답에서 데이터 추출
soup = BeautifulSoup(html, "html.parser")

rows = []
data = []

trs = soup.find_all("tr")

for tr in trs[10:]:
    tds = tr.find_all("td")
    team_id = tds[1].find('a').attrs['href'].split('&')[1].split('=')[1]
    tds = [td.text for td in tds]
    tds.append(team_id)
    rows.append(tds)

for row in rows:
    temp = []
    for index in range(1, len(row)):
        temp.append(row[index])
    data.append(temp)

In [21]:
team_defense_df = pd.DataFrame(data, columns=['팀', '시즌', 'WAAwithPOS', 'G', 'GS', 'IP', 'TC', 'PO', 'Ass', 'E', 'F%', 'RF9', 'Range RAA', 'Arm RAA', 'Err RAA', 'DP RAA', 'Bt RAA', 'CS RAA', 'Blk RAA', 'Frm RAA', 'Total RAA', '/144', 'POSAdj', 'RAAwithPOS', 'WAAwoPOS', 'DUMMY', 'T_CODE'])

team_defense_df = team_defense_df.dropna(axis=0)

team_defense_df.head()

,팀,시즌,WAAwithPOS,G,GS,IP,TC,PO,Ass,E,F%,RF9,Range RAA,Arm RAA,Err RAA,DP RAA,Bt RAA,CS RAA,Blk RAA,Frm RAA,Total RAA,/144,POSAdj,RAAwithPOS,WAAwoPOS,DUMMY,T_CODE
0,KT,19,6.409,2115,1321,11888.2,5246,3781,1372,93,98.2,3.90,77.63,-0.61,-4.01,0.00,-0.55,0.00,0.00,4.03,76.50,5.21,-10.04,66.46,7.487,6.409,12001
1,두산,19,4.194,1987,1349,12056.0,5232,3813,1345,74,98.6,3.85,28.76,-1.19,3.96,0.00,-0.12,0.00,0.00,20.05,51.47,3.73,-9.23,42.08,5.037,4.194,6002
2,SSG,22,4.135,2069,1325,12120.2,5283,3853,1333,97,98.2,3.85,48.06,3.79,3.24,0.00,-0.67,0.00,0.00,-2.31,52.12,3.63,-9.50,42.61,5.088,4.135,9002
3,KT,23,3.512,2177,1304,11884.0,5127,3805,1242,80,98.4,3.82,32.35,1.09,6.80,0.00,0.48,0.00,0.00,3.73,44.45,2.94,-7.92,36.53,4.303,3.512,12001
4,KT,20,2.883,2435,1363,12371.2,5368,3845,1426,97,98.2,3.83,36.75,2.85,-0.34,0.00,-0.61,0.00,0.00,4.01,42.66,2.52,-9.81,32.85,3.763,2.883,12001
5,LG,22,2.790,2196,1328,12135.2,5216,3837,1299,80,98.5,3.81,41.06,-1.02,4.04,0.00,-0.13,0.00,0.00,-6.15,37.80,2.48,-8.85,28.94,3.691,2.790,5002
6,LG,19,2.573,2041,1328,11972.1,5278,3788,1414,76,98.6,3.91,24.18,1.63,1.99,0.00,0.12,0.00,0.00,6.79,34.71,2.45,-8.41,26.29,3.397,2.573,5002
7,KT,22,2.567,2235,1345,12180.0,5254,3824,1341,89,98.3,3.82,33.16,-0.19,-0.32,0.00,-0.75,0.00,0.00,3.80,35.70,2.30,-8.82,26.71,3.486,2.567,12001
8,삼성,20,2.449,2015,1328,11762.0,5055,3773,1191,91,98.2,3.80,29.12,-0.37,1.01,0.00,-0.10,0.00,0.00,6.70,36.36,2.60,-8.32,28.04,3.207,2.449,1001
9,한화,21,2.174,1972,1339,11850.0,5029,3731,1186,112,97.8,3.73,31.40,-2.71,0.87,0.00,1.34,0.00,0.00,-4.16,26.74,1.95,-3.98,22.76,2.558,2.174,7002


In [22]:
table_name = 'df_team_defense'

with engine.connect() as con:
    team_defense_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )

In [5]:
url = 'https://statiz.sporki.com/stats/?m=team&m2=all&m3=default&so=WAR&ob=DESC&year=2024&sy=2019&ey=2024&te=&po=&lt=&reg=&pe=&ds=&de=&we=&hr=&ha=&ct=&st=&vp=&bo=&pt=&pp=&ii=&vc=&um=&oo=&rr=&sc=&bc=&ba=&li=&as=&ae=&pl=&gc=&lr=&pr=50&ph=&hs=&us=&na=&ls=&sf1=&sk1=&sv1=&sf2=&sk2=&sv2='

response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

res = requests.get(url, headers=headers)
res.raise_for_status()

html = response.text

# 응답에서 데이터 추출
soup = BeautifulSoup(html, "html.parser")

rows = []
data = []

trs = soup.find_all("tr")

for tr in trs[10:]:
    tds = tr.find_all("td")
    team_id = tds[1].find('a').attrs['href'].split('&')[1].split('=')[1]
    tds = [td.text for td in tds]
    tds.append(team_id)
    rows.append(tds)

for row in rows:
    temp = []
    for index in range(1, len(row)):
        temp.append(row[index])
    data.append(temp)

In [7]:
team_total_df = pd.DataFrame(data, columns=['팀', '시즌', 'WAR', 'DUMMY', '평균 대비 타격 승리 기여도', '평균 대비 도루 승리 기여도', '평균 대비 주루 승리 기여도', '평균 대비 수비 승리 기여도', '대체 승리', '대체 포지션 조정', '평균 대비 선발 승리 기여도', '평균 대비 불펜 승리 기여도', '선발 대체 승리', '불펜 대체 승리', '선발 WAR', '불펜 WAR', 'T_CODE'])

team_total_df.head()

,팀,시즌,WAR,DUMMY,평균 대비 타격 승리 기여도,평균 대비 도루 승리 기여도,평균 대비 주루 승리 기여도,평균 대비 수비 승리 기여도,대체 승리,대체 포지션 조정,평균 대비 선발 승리 기여도,평균 대비 불펜 승리 기여도,선발 대체 승리,불펜 대체 승리,선발 WAR,불펜 WAR,T_CODE
0,LG,22,62.98,62.98,9.15,-0.18,0.45,-0.93,2.42,-0.04,2.80,5.92,12.39,4.31,15.01,10.09,5002
1,NC,20,61.56,61.56,9.67,0.17,0.21,2.86,2.23,0.33,6.02,0.34,12.88,4.33,18.44,4.76,11001
2,키움,19,61.39,61.39,10.80,0.45,-0.09,8.07,2.45,-0.19,5.20,5.79,13.17,3.66,18.38,9.42,10001
3,두산,19,60.23,60.23,9.70,0.18,0.18,-2.95,2.46,-0.35,3.69,-0.93,14.10,3.30,17.47,2.47,6002
4,LG,23,59.18,59.18,13.03,-1.30,0.19,-0.35,2.39,-0.33,1.32,3.94,12.01,4.36,13.50,8.33,5002


In [8]:
table_name = 'df_team_total'

with engine.connect() as con:
    team_total_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )

In [6]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}
data = []

for year in range(2019, 2025):
    url = f'https://statiz.sporki.com/season/?m=teamoverall&year={year}'

    response = requests.post(url, headers=headers)  # 또는 requests.get() 사용

    res = requests.get(url, headers=headers)
    res.raise_for_status()

    html = response.text

    # 응답에서 데이터 추출
    soup = BeautifulSoup(html, "html.parser")

    if year == 2024:
        items = soup.select_one(".table_type03")
        trs = items.find_all("tr")

    else:
        items = soup.select(".table_type03")
        items = items[1]
        trs = items.find_all("tr")

    rows = []

    for tr in trs[1:]:
        tds = tr.find_all("td")
        team_id = tds[1].find('a').attrs['href'].split('&')[1].split('=')[1]
        tds = [td.text for td in tds]
        tds.append(team_id)
        rows.append(tds)

    for row in rows:
        temp = []
        for index in range(0, len(row)):
            temp.append(row[index])
        temp.insert(2, year)
        data.append(temp)


In [9]:
team_rank_df = pd.DataFrame(data, columns=['순위', '팀', '연도', '게임수', '승', '무', '패', '승차', '승률', 'T_CODE'])

team_rank_df.head()

,순위,팀,연도,게임수,승,무,패,승차,승률,T_CODE
0,1,SK,2019,144,88,1,55,0.0,0.615,9001
1,1,두산,2019,144,88,1,55,0.0,0.615,6002
2,3,키움,2019,144,86,1,57,2.0,0.601,10001
3,4,LG,2019,144,79,1,64,9.0,0.552,5002
4,5,NC,2019,144,73,2,69,14.5,0.514,11001
5,6,KT,2019,144,71,2,71,16.5,0.500,12001
6,7,KIA,2019,144,62,2,80,25.5,0.437,2002
7,8,삼성,2019,144,60,1,83,28.0,0.420,1001
8,9,한화,2019,144,58,0,86,30.5,0.403,7002
9,10,롯데,2019,144,48,3,93,39.0,0.340,3001


In [ ]:
table_name = 'df_team_rank'

with engine.connect() as con:
    team_rank_df.to_sql(
        name = table_name.lower(),
        con = con,
        if_exists = if_exists
    )